### Importing necessary libraries

In [1]:
import re,time
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
import numpy as np
import json
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')
from queue import PriorityQueue
generator = pipeline("text-generation", model="gpt2")

### Context + Q/A USING BERT,PIPELINE,VECTORS, MiniLM-L6-SENTENCE_TRANSFORMER

In [9]:
start_time=time.time()
sen=input()
test_vec1=model.encode([sen])[0]
filename = 'data.json'
with open(filename, 'r') as json_file:
    data_loaded = json.load(json_file)
hashmap={}
for i in range(3041):
    test_vec3=np.array(data_loaded[str(i)]['vector'])
    similarity_score = 1-distance.cosine(test_vec1,test_vec3)
    if(similarity_score>0.63):
        m=data_loaded[str(i)]['question'].strip()
        n=data_loaded[str(i)]['answer'].strip()
        hashmap[m+' '+n]=similarity_score
s=''
count=7
for i in hashmap:
    s=s+i[0]+'\n'
    if(count<=0):
        break
    count-=1
    
    
with open("data_context.json","r") as outfile:
    saved_data=json.load(outfile)
que=sen
for i in range(len(saved_data)):
    test_vec2=np.array(saved_data[str(i)]['vector'])
    similarity_score=1-distance.cosine(test_vec2,test_vec1)
    if(similarity_score>0.60):
        hashmap[saved_data[str(i)]['sentence'].strip()]=similarity_score
hashmap = sorted(hashmap.items(), key=lambda item: item[1] ,reverse=True)
count=7
# print(hashmap)
for i in hashmap:
    s=s+i[0]+'\n'
    if(count<=0):
        break
    count-=1
# print(s)
if(len(s)==0):
    print("Sorry i cannot answer that!")
# Define your context and combined question
else:
    sequence_to_remove = "W\nW\nW\nW\nW\nW\nW\nW\n"

    # Check if the sequence is present and replace it
    if sequence_to_remove in s:
        s = s.replace(sequence_to_remove, "")
    else:
        s= s
    context = s.lower()
    question = que.lower()
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

    # Answer the question based on the contet
    result = qa_pipeline(question=question, context=context)

    # Print the result
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")
    print("\n\n\n Total time:",time.time()-start_time)

who was the first player to take a odi hat-trick


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: who was the first player to take a odi hat-trick
Answer: jalaluddin



 Total time: 116.56649041175842


In [10]:
print(s)

W
W
W
W
1
D
W
S
Who took the first ODI hat-trick? Jalaluddin
Who became the first English bowler to take a hat-trick in an ODI? James Anderson
Who was the first cricketer to take a hat-trick in a World Cup ODI match? Chetan Sharma, India
Who is the first ever cricketer to take a hat trick in a ODI? Jalaluddin
Who took th first ODI hat-trick? Jalaluddin
Who was the bowler that took the first-ever ODI hat-trick? Jalal ud-din
Who was the first player in Test cricket to achieve a Hat-trick? Fred Spoffoth (AUS)
Who was the first cricketer to take a hat-trick in Test cricket? Fred Spofforth



In [25]:
import random
pre=["Here's what I found:","Based on my research:","According to source:","Here's the answer to your question:","The answer is:","In summary, it is:","To sum up, it's:","Here's the information you requested:"]
suf=["I'm always available to help.","Don't hesitate to reach out for more assistance.","If you need clarification, don't hesitate to ask.","I'm here to support you with any further inquiries.",    "Feel free to ask if you need more details.","Let me know if you have any further questions.","If you need more information, just ask.","I'm here to help with any additional queries."]
print(f"Answer: {pre[random.randint(0,len(pre)-1)]}+' '+{result['answer']}+'\n'+{suf[random.randint(0,len(suf)-1)]}")

Answer: The answer is:+' '+w
w
w
w
taylor+'
'+Let me know if you have any further questions.


In [8]:
print(hashmap)

[('Taylor was the first bowler to take a hat-trick in the Champions Trophy so far', 0.7737929787416462), ('Who became the first English bowler to take a hat-trick in an ODI? James Anderson', 0.6481041309567812), ('Who was the bowler that took the first-ever ODI hat-trick? Jalal ud-din', 0.6405409108029112), ('Who was the first bowler to take a hat-trick in test cricket? Spofforth', 0.6387893636609259), ('Who was the first bowler to take a hat-trick in IPL history? Lakshmipathy Balaji', 0.6381210122401597)]


## Loading a pre_trained Bert model which has already been trained on SQUAD Dataset

In [3]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import warnings
warnings.simplefilter("ignore")

weight_path = "kaporter/bert-base-uncased-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(weight_path)
model = BertForQuestionAnswering.from_pretrained(weight_path)

In [6]:
print(len(s))

1930


In [140]:
len(s)/1000

0.61

In [20]:
question = question+' the answer is '
#print(len(s))
l=[]
length=int(1+len(s)/1000)
for i in range(length):
    if(i==length-1):
        context=s[1000*i:]
    else:
        context=s[i*1000:(i+1)*1000]
    #context = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."
    #context="Sunrisers Hyderabad made their IPL debut in the 2013 season.[4] They retained 20 players from the Chargers, which left slots open for 13 players (eight Indian, five overseas). They filled six of these with Thisara Perera, Darren Sammy, Sudeep Tyagi, Nathan McCullum, Quinton de Kock and Clint McKay. Kumar Sangakkara captained SRH for nine matches and Cameron White was captain for the remaining seven, as well as the eliminator match in the playoffs.[15] In their inaugural season, the team reached the playoffs but were eliminated after losing against Rajasthan Royals by 4 wickets at Feroz Shah Kotla in Delhi on 22 May 2013.[16] The team played all of their home games in Hyderabad."
    #context="On 13 September 2007,[25] following India's victory at the 2007 T20 World Cup,[26] the BCCI announced a franchise based Twenty20 cricket competition known as the Indian Premier League. The inaugural season was scheduled to start in April 2008, commencing with a high-profile ceremony in New Delhi. BCCI Vice-president Lalit Modi, who led the IPL initiative, provided details of the tournament, including its format, prize money, franchise revenue system, and squad composition rules. The league, to be managed by a seven-man governing council, would also serve as the qualifying mechanism for that year's Champions League Twenty20.[25]To determine team ownership, an auction for the franchises was held on 24 January 2008. The reserve prices for the eight franchises totalled $400 million,[24] but the auction ultimately raised $723.59 million.[27] The league officially commenced in April 2008, featuring Chennai Super Kings (CSK), Mumbai Indians (MI), Delhi Daredevils (DD), Kings XI Punjab (KXIP), Deccan Chargers (DC), Rajasthan Royals (RR), Kolkata Knight Riders (KKR), and Royal Challengers Bangalore (RCB).[28]"
    input_ids = tokenizer.encode(question, context)
    #print (f'We have about {len(input_ids)} tokens generated')

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # print(" ")
    # print('Some examples of token-input_id pairs:')
    #for i, (token,inp_id) in enumerate(zip(tokens,input_ids)):
        #print(token,":",inp_id)
    sep_idx = tokens.index('[SEP]')

    token_type_ids = [0 for i in range(sep_idx+1)] + [1 for i in range(sep_idx+1,len(tokens))]
    #print(token_type_ids)
    out = model(torch.tensor([input_ids]),
                    token_type_ids=torch.tensor([token_type_ids]))

    start_logits,end_logits = out['start_logits'],out['end_logits']
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits)

    ans = ''.join(tokens[answer_start:answer_end+1])
    if ans=='' or ans=='[CLS]':
        l=l
    else:
        s3=''
        for i in ans:
            if i!='#':
                s3+=i
        l.append(s3)
l = sorted(l, key=lambda x: (l.count(x), x))
l=l[::-1]
if(len(l)==0):
    print("Sorry i have not been trained on this topic")
else:
    print('predicted answer is:-',l[0])

predicted answer is:- goldenduck


In [21]:
print(l)

['goldenduck', 'goldenduck']


## NLP to find similarities between two sentences

In [4]:
from scipy.spatial import distance
import re,time
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence1="Houghton scored 84 against Australia, making a total of 176 runs in the world cup in 1983.  Curran scored the most runs in total - 212.  His top score was 73.  Fletcher scored 191 altogether, with a highest of 71*."
sentence2="how many total runs did curran score"
print("Sentence 1:",sentence1)
test_vec1=model.encode([sentence1])[0]
# print("Vector of this sentence is: ",test_vec1)
print("*******************************************************************************************")
print("Sentence 2:",sentence2)
test_vec2=model.encode([sentence2])[0]
# print("Vector of this sentence is: ",test_vec2)
similarity_score = 1-distance.cosine(test_vec1,test_vec2)
print("Similarity score is:",similarity_score)
print("Total time taken is ",time.time()-start)

Sentence 1: Houghton scored 84 against Australia, making a total of 176 runs in the world cup in 1983.  Curran scored the most runs in total - 212.  His top score was 73.  Fletcher scored 191 altogether, with a highest of 71*.
*******************************************************************************************
Sentence 2: how many total runs did curran score
Similarity score is: 0.7119457125663757
Total time taken is  2.9110498428344727


In [90]:
start=time.time()
similarity_score = 1-distance.cosine(test_vec1,test_vec3)
print(similarity_score)
print(time.time()-start)

0.09644113919184283
0.0010476112365722656


In [5]:
from scipy.spatial import distance
import re,time
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
s=time.time()
# Sample sentence
test =  "When was world championship started"
# print('Test sentence:',test)
test_vec = model.encode([test])[0]
# print(test_vec)
for i in range(100):
    similarity_score = 1-distance.cosine(test_vec, test_vec1)
#     print(f'\nFor {sent}\nSimilarity Score = {similarity_score} ')
print((-s+time.time()))

0.11793351173400879


In [156]:
with open('modified_questions.json', 'r') as file:
    saved_file = json.load(file)

In [160]:
i=0
start_time=time.time()
question= "At the start of each cricket match, a coin is tossed by the team captains. Whoever wins the toss always choose to bat, to ensure the runs are on the board in case it starts to rain."
for obj in saved_file:
    x=compare(question,obj['qas'][0]['question'])
    if(x>0.9):
        break
print('Total time taken :- ',1000*(time.time()-start_time),'ms')
# s=' '.join(h)
#print(s)


For Cricket legend Don Bradman, and considered by many to be the best our sport has seen, was born in which city?
Similarity Score = 0.3029899597167969 

For In which year did the Invincibles tour England?
Similarity Score = 0.18595144152641296 

For A record for the longest test wining streak was set in 2001.  How many was the streak?
Similarity Score = 0.15594632923603058 

For Which team broke Australia's winning streak in 2001, and which match of the test series was this (HINT: It was a 3 test match series)?
Similarity Score = 0.3259376585483551 

For Before Matthew Hayden broke the record for the highest Individual Test score with 380, Brian Lara had the record.  Which was his score?
Similarity Score = 0.1884753704071045 

For Don Bradman died on the 13th of September 2001?
Similarity Score = 0.01241418719291687 

For Australia won their first ever Test Match against England?
Similarity Score = 0.2945070266723633 

For Who won the Inaugural World Cup?
Similarity Score = 0.2010194


For What is an LBW?
Similarity Score = 0.15194807946681976 

For Which of these cricket players set a record for the most World Cup gold medals?
Similarity Score = 0.3260113596916199 

For Where did the 2003 Cricket World Cup take place?
Similarity Score = 0.44770219922065735 

For Where did the 2007 Cricket World Cup take place, and who won it?
Similarity Score = 0.443708211183548 

For What crime was the South African captain Hansie Cronje accused of?
Similarity Score = 0.1421816200017929 

For How did Hansie Cronje die?
Similarity Score = 0.028273381292819977 

For What country did Raymond Price play for?
Similarity Score = 0.05942093953490257 

For Who got the most runs in the 438 game, and how many did they get?
Similarity Score = 0.21140052378177643 

For Who was Man of the Tournament in the 2007 ICC Cricket World Cup?
Similarity Score = 0.4110850393772125 

For Which two players had the highest scoring partnership at the Galle International Stadium, Sri Lanka in 2000?
Similarit


For A is for Ashes. The test matches played between two nations are referred to as The Ashes. Which are these two teams?
Similarity Score = 0.4718082845211029 

For B is for Bails, as essential as the stumps, bat and ball. How many bails are required to be used in a game of cricket?
Similarity Score = 0.4557901620864868 

For C is for Chin music. What is chin music in cricket?
Similarity Score = 0.39476484060287476 

For D is for Declare. What does it mean when a captain declares?
Similarity Score = 0.2721758484840393 

For E is for Extra. Which of the following is NOT considered as an extra run in cricket?
Similarity Score = 0.43653976917266846 

For F is for Flight. In cricket what does this term mean?
Similarity Score = 0.40172550082206726 

For G is for Groundsman. Who is a groundsman in cricket?
Similarity Score = 0.4459921419620514 

For H is for Hawk Eye, a technology used in cricket. For what purpose is this technology used?
Similarity Score = 0.3153499364852905 

For I is for


For The bowler has just bowled six consecutive legal deliveries, the final one has become "dead" by being gathered safely in the wicketkeeper's gloves, the batsmen have no intention of taking a run. What will you hear the umpire calling at this stage in the proceedings?
Similarity Score = 0.4375797510147095 

For The bowler bowls, the batsman fends at the ball, it goes through to the wicketkeeper and there is an appeal to the umpire for a catch. What sound has the wicketkeeper heard?
Similarity Score = 0.46011582016944885 

For It's a Saturday afternoon, and the first innings has been going along for two or three hours. What welcome sound might some of the fielders hear from the pavilion?
Similarity Score = 0.4707249402999878 

For Whilst walking through the English countryside on a summer's evening, a familiar sound drifts over the gentle breeze to your ears. A game of cricket is taking place. What sound do you hear?
Similarity Score = 0.39184102416038513 

For Between every ball, yo


For What is the name of the fielding position just behind and to the off side of the keeper?
Similarity Score = 0.45341166853904724 

For What does it mean when a batsmen 'walks'?
Similarity Score = 0.5119468569755554 

For What is a 'hat trick'?
Similarity Score = 0.33578041195869446 

For What is a 'yorker'?
Similarity Score = 0.07685894519090652 

For What is meant when we talk of a 'duck'?
Similarity Score = 0.141513854265213 

For What is referred to by the term 'maiden over'?
Similarity Score = 0.09906395524740219 

For How many deliveries are there in a standard over?
Similarity Score = 0.032047200947999954 

For What is a 'beamer'?
Similarity Score = 0.09992906451225281 

For I've just been bowled a "beamer." What part of my body should I be worried about?
Similarity Score = 0.14730168879032135 

For I thought we were all playing cricket, but the commentator has just said the batsman was "fishing." What could he mean?
Similarity Score = 0.38597869873046875 

For What type of d

### Loading the dataset and saving in dictionary format

In [18]:
import json
with open("modified_questions.json","r") as f:
    saved_file=json.load(f)
i=0
d={}
start_time=time.time()
for obj in saved_file:
    x=obj['qas'][0]['question'] #input
    test_vec=model.encode([x])[0]
    d[i]={'vector':test_vec,'question':obj['qas'][0]['question'],'answer':obj['qas'][0]['answers'][0]['text'],'context':obj['context']}
    i+=1
print("Total time taken: ",time.time()-start_time)
print(d)

Total time taken:  1271.7684094905853


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import json
with open("modified_questions.json","r") as outfile:
    saved_data=json.load(outfile)
j=0
d={}
for obj in saved_data:
    context=obj['context']
    l=context.split('.')
    for i in l:
        sen=i
        test_vec=model.encode([sen])[0]
        d[j]={'vector':test_vec,'sentence':sen}
        j+=1
print(len(d))

In [19]:
print(len(d))

3041


##### format of the dictionary

In [33]:
print(d[3040])

{'vector': array([-3.08824982e-03,  7.33531341e-02, -5.83087951e-02,  6.69139251e-03,
       -3.79388011e-03,  1.42375929e-02, -2.77687777e-02, -3.81168816e-03,
       -1.07546514e-02,  9.50805321e-02,  1.02501130e-02, -6.59564286e-02,
        2.04474442e-02,  4.78838347e-02,  7.80324116e-02, -2.67943647e-02,
       -2.47644614e-02, -8.32411125e-02,  9.59164575e-02, -5.67160174e-02,
       -4.20354353e-03,  4.38526049e-02, -5.69773614e-02, -6.83731586e-02,
        4.67091501e-02, -4.45830747e-02, -1.08067524e-02,  4.60730977e-02,
       -7.71872234e-03, -2.58482583e-02,  6.40874803e-02, -8.58865399e-03,
        2.36415658e-02,  9.74896848e-02, -4.00319360e-02, -5.92983626e-02,
       -1.06121330e-02,  3.76376547e-02,  1.75748728e-02, -1.05905645e-02,
       -3.11648101e-03, -5.90406358e-02,  6.30478933e-02, -4.75150980e-02,
        2.25662068e-03,  3.47688794e-02,  3.59817035e-02,  7.89076686e-02,
        3.59743647e-02,  8.21396150e-03,  5.60758971e-02,  4.86046635e-02,
       -2.9854

In [36]:
print(type(d))

<class 'dict'>


### saving above dictionary into a json

In [37]:
import numpy as np
import json

class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)
json_data = json.dumps(d, cls=NumpyArrayEncoder)

print(json_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
#loading saved data
with open("data_context.json", "r") as outfile: 
    saved_file=json.load(outfile)

In [17]:
print(len(saved_file["200"]["vector"]))

384


In [34]:
print(time.time())

1717079161.5571773


## Predicting answers using Retrieveal Augumented Generation(RAG) and BERT Model

In [89]:
start_time=time.time()
sen=input()
test_vec1=model.encode([sen])[0]#Converting input text to vector
filename = 'data.json'# loading the dataset containing vectors
with open(filename, 'r') as json_file:
    data_loaded = json.load(json_file)
hashmap={}
for i in range(3041):
    test_vec3=np.array(data_loaded[str(i)]['vector'])
    similarity_score = 1-distance.cosine(test_vec1,test_vec3)
    if(similarity_score>0.75):
        m=data_loaded[str(i)]['question'].strip()
        n=data_loaded[str(i)]['answer'].strip()
        hashmap[m+' '+n]=similarity_score
s=''#saving relevant context into a string
count=7
for i in hashmap:
    s=s+i[0]+'\n'
    if(count<=0):
        break
    count-=1
    
    
with open("data_context.json","r") as outfile:#repeating above procedure for another vector file
    saved_data=json.load(outfile)
que=sen
for i in range(len(saved_data)):
    test_vec2=np.array(saved_data[str(i)]['vector'])
    similarity_score=1-distance.cosine(test_vec2,test_vec1)
    if(similarity_score>0.61):
        hashmap[saved_data[str(i)]['sentence'].strip()]=similarity_score
hashmap = sorted(hashmap.items(), key=lambda item: item[1] ,reverse=True)
count=7
# print(hashmap)
for i in hashmap:
    s=s+i[0]+'\n'
    if(count<=0):
        break
    count-=1
# print(s)
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

# Load the pipeline for text generation (GPT-2 as an example)
generator = pipeline("text-generation", model="gpt2")

# Define your context and combined question
context = s.lower()
question = que.lower()
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Answer the question based on the context
result = qa_pipeline(question=question, context=context)


print(f"Question: {question}")
print(f"Answer: {result['answer']}")
print("\n\n\n Total time:",time.time()-start_time)

what is hawk eye in cricket


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: what is hawk eye in cricket
Answer: a technology used mainly in leg before wicket decisions



 Total time: 30.54766607284546
